# SETUP

In [1]:
# This automates data tabulation onto google sheets 

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build

import os

# new directory path
new_directory = '/Users/levan/ATENEO MASTERAL/Thesis'

# Change the current working directory
os.chdir(new_directory)

# Use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('thesis-432315-12daec8d1ff6.json', scope)

service = build('sheets', 'v4', credentials=creds)

client = gspread.authorize(creds)

spreadsheet_id = '18Hi92aObP2ST7OrpVlsH8C8j0KS05a93sDRsMN0ZFPM' 

# SOFT Voting

## Load Dataset

In [2]:
import os

# Specify the new directory path
new_directory = '/Users/levan/ATENEO MASTERAL/Thesis/Development'

# Change the current working directory
os.chdir(new_directory)

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AutoConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Adjust file paths to your local system
file_path = 'Corpus/data_b.csv'

df = pd.read_csv(file_path)

# Split the data into training and test sets (70-30 split)
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# Extract texts and labels
train_texts = train_df['text'].tolist()
train_labels = train_df['label'].tolist()

test_texts = test_df['text'].tolist()
test_labels = test_df['label'].tolist()

## Load Models and Tokenizers

In [4]:
def load_model_and_tokenizer(model_path, tokenizer_path, base_model):
    # Load the tokenizer from the local directory
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    
    # Load the configuration from the base model, then update configuration if needed
    config = AutoConfig.from_pretrained(base_model, num_labels=2)

    # Initialize the model with the configuration
    model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config)

    # Ensure the model is in evaluation mode
    model.eval()
    
    return model, tokenizer

model_info = {   
    
    'HateBERT': {
        'model_path': 'BERT models/hs_HateBERT-finetuned',
        'tokenizer_path': 'BERT models/hs_HateBERT-finetuned',
        'base_model': 'GroNLP/hateBERT'
    },

    'TagBERT': {
        'model_path': 'BERT models/hs_Tag-Roberta-finetuned',
        'tokenizer_path': 'BERT models/hs_Tag-Roberta-finetuned',
        'base_model': 'jcblaise/roberta-tagalog-base'
    },

    'DeBERTa': {
        'model_path': 'BERT models/hs_DeBERTa-finetuned',
        'tokenizer_path': 'BERT models/hs_DeBERTa-finetuned',
        'base_model': 'microsoft/deberta-v3-base'
    },
}

models_and_tokenizers = {name: load_model_and_tokenizer(info['model_path'], 
                                                        info['tokenizer_path'], 
                                                        info['base_model']) 
                         for name, info in model_info.items()}

## Apply Tokenization

In [5]:
def texts_to_dataloader(texts, tokenizer, batch_size=32):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

    # Move tensors to the specified device
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)
    
    dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'])
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

## Perform Soft Voting and Prediction

In [6]:
%load_ext memory_profiler

In [7]:
%%memit

def soft_voting_predict(models_and_tokenizers, texts):
    sum_probs = None
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    
    # Loop through each model and its corresponding tokenizer
    for name, (model, tokenizer) in models_and_tokenizers.items():
        
        # Explicitly move each model to the MPS device if available
        model.to(device)
        
        dataloader = texts_to_dataloader(texts, tokenizer)
        model_probs_list = []
        for batch in dataloader:
            input_ids, attention_mask = batch
            input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
            
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
                model_probs_list.append(probs.cpu().numpy())
        
        model_probs = np.concatenate(model_probs_list)
        
        if sum_probs is None:
            sum_probs = model_probs
        else:
            sum_probs += model_probs

    # Average the probabilities
    avg_probs = sum_probs / len(models_and_tokenizers)
    
    # Perform voting based on highest average probabilities
    final_preds = np.argmax(avg_probs, axis=-1)
    return final_preds

# Perform inference and voting
final_predictions = soft_voting_predict(models_and_tokenizers, test_texts)


peak memory: 2225.23 MiB, increment: 147.70 MiB


## Evaluate Model

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Calculate precision, recall, accuracy, and F1 score
precision = precision_score(test_labels, final_predictions, average='binary')  # Adjust 'binary' as needed
recall = recall_score(test_labels, final_predictions, average='binary')  # Adjust 'binary' as needed
accuracy = accuracy_score(test_labels, final_predictions)  # Use the original test_labels list
f1 = f1_score(test_labels, final_predictions, average='binary')  # Adjust 'binary' as needed

# Generate confusion matrix
conf_matrix = confusion_matrix(test_labels, final_predictions)

# Print the metrics
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Ensemble accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

Precision: 0.7636068237205524
Recall: 0.8223972003499562
Ensemble accuracy: 0.801845166466105
F1 Score: 0.7919123841617524
Confusion Matrix:
[[1059  291]
 [ 203  940]]


In [9]:
# SAVE TO GOOGLE SHEET

# Define the range and values to update
range_name = '3-3!B3:E3'  

values = [[
    f"{precision * 100:.2f}",
    f"{recall * 100:.2f}",
    f"{accuracy * 100:.2f}",
    f"{f1 * 100:.2f}"
]]

# Prepare the request body
body = {
    'values': values
}

# Call the Sheets API to update the values
result = service.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id, 
    range=range_name,
    valueInputOption='USER_ENTERED',
    body=body
).execute()

print('Updated cells count:', result.get('updatedCells'))

Updated cells count: 4


# Validate on Data C

## Load Dataset

In [10]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Load validation data
validation_file_path = 'Corpus/data_c.csv'
validation_df = pd.read_csv(validation_file_path)


# Prepare the validation texts and labels
validation_texts = validation_df['text'].tolist()
validation_labels = validation_df['label'].values  



## Perform Soft Voting

In [11]:
%%memit
# Perform inference and voting on the validation texts
validation_predictions = soft_voting_predict(models_and_tokenizers, validation_texts)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


peak memory: 2303.19 MiB, increment: 137.30 MiB


## Evaluate Model

In [12]:
# Calculate precision, recall, accuracy, and F1 score
precision = precision_score(validation_labels, validation_predictions, average='binary')  
recall = recall_score(validation_labels, validation_predictions, average='binary')  
accuracy = accuracy_score(validation_labels, validation_predictions)
f1 = f1_score(validation_labels, validation_predictions, average='binary')  

# Generate confusion matrix
conf_matrix = confusion_matrix(validation_labels, validation_predictions)

# Print the metrics
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Validation accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

Precision: 0.7722342733188721
Recall: 0.827906976744186
Validation accuracy: 0.8061721710882512
F1 Score: 0.7991021324354658
Confusion Matrix:
[[777 210]
 [148 712]]


In [13]:
# SAVE TO GOOGLE SHEET

# Define the range and values to update
range_name = '3-3!F3:I3'  

values = [[
    f"{precision * 100:.2f}",
    f"{recall * 100:.2f}",
    f"{accuracy * 100:.2f}",
    f"{f1 * 100:.2f}"
]]

# Prepare the request body
body = {
    'values': values
}

# Call the Sheets API to update the values
result = service.spreadsheets().values().update(
    spreadsheetId=spreadsheet_id, 
    range=range_name,
    valueInputOption='USER_ENTERED',
    body=body
).execute()

print('Updated cells count:', result.get('updatedCells'))

Updated cells count: 4


In [14]:
import altair as alt
import pandas as pd
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(validation_labels, validation_predictions)

# Define class names
class_names = ['Hate Speech', 'Non Hatespeech']

# Convert confusion matrix to DataFrame
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names).reset_index().melt(id_vars='index')
cm_df.columns = ['True', 'Predicted', 'Count']

# Ensure the order of categories
cm_df['True'] = pd.Categorical(cm_df['True'], categories=class_names, ordered=True)
cm_df['Predicted'] = pd.Categorical(cm_df['Predicted'], categories=class_names, ordered=True)

# Create the Altair plot
heatmap = alt.Chart(cm_df).mark_rect().encode(
    x=alt.X('Predicted:O', sort=class_names),
    y=alt.Y('True:O', sort=class_names),
    color='Count:Q',
    tooltip=['True', 'Predicted', 'Count']
).properties(
    width=400,
    height=300,
    title='3 Soft Voting-3'
)

# Add text labels
text = heatmap.mark_text(
    align='center',
    baseline='middle',
    fontSize=12
).encode(
    text='Count:Q',
    color=alt.condition(
        alt.datum.Count > cm.max() / 2,
        alt.value('white'),
        alt.value('black')
    )
)

# Combine heatmap and text
final_chart = heatmap + text

# Display the plot
final_chart.show()

alt.LayerChart(...)

In [15]:
# Specify the folder path
folder_path = os.path.expanduser('Results/Ensemble Model Results/On Hatespeech dataset/OH Using HS_Finetuned Models/Soft Voting/')

# Save the plot using vl-convert
file_path_png = os.path.join(folder_path, '3SV-3.png')
final_chart.save(file_path_png)

print(f"Plot saved to {file_path_png}")

Plot saved to Results/Ensemble Model Results/On Hatespeech dataset/OH Using HS_Finetuned Models/Soft Voting/3SV-3.png
